# 세미프로젝트 파일정리

1. 22년 분기(1~4), 서울시 자치구, 외식 업종, 매출등급 전처리 
2. 위 데이터(키)와 개폐업, 생존율, 최신영업기간 데이터 결합 및 전처리
3. 위 데이터(키,개폐업 등)와 매출 데이터 결합 및 전처리
4. 타입변경, 문자제거 전처리
5. 탐색적 데이터 분석 및 시각화 (EDA)
6. 머신러닝 적용을 위한 주요 데이터 인코딩 전처리
7. 머신러닝 모델링 

# 3. 매출 데이터 결합 및 전처리

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [2]:
from matplotlib import pyplot
from matplotlib import font_manager

f_path = "C:\Windows\Fonts\Malgun.ttf"
font_manager.FontProperties(fname=f_path).get_name()

from matplotlib import rc
rc('font', family='Malgun Gothic')

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='white', color_codes=True)
sns.set_style('whitegrid', {'font.family':'Malgun Gothic'})
import matplotlib.ticker as mticker

In [3]:
추정매출 = pd.read_csv("./서울시 상권분석서비스(상권-추정매출2022).csv", dtype = {'분기당_매출_금액' : 'int64'},  encoding = "ansi")
생활인구 = pd.read_csv("./서울시 상권분석서비스(상권-생활인구2022).csv", encoding = "ansi")
직장인구 = pd.read_csv("./서울시 상권분석서비스(상권-직장인구2022).csv", encoding = "ansi")
상권영역 = pd.read_csv("./서울시 상권분석서비스(상권영역).csv", encoding = "ansi")
법정동코드 = pd.read_csv("./서울시 건축물대장 법정동 코드정보.csv", encoding = "ansi")
매출등급 = pd.read_csv("./필지단위 소상공인 매출등급 정보.csv", encoding = "ansi")
코드변환 = pd.read_csv("./A_CS 코드변환.txt")
분기_업종_구_등급 = pd.read_csv("./년도_업종_구_등급.csv", encoding = "ansi")


# 전처리 후 csv 추가 파일
등급_개폐업율추가 = pd.read_csv("등급_개폐업율추가.csv", encoding = "ansi")

In [4]:
등급_개폐업율추가

,Unnamed: 0,데이터기준년월,시군구명,서비스업종코드,서비스업종,매출등급,매출등급01,개업수,폐업수,개업률,폐업률,1년생존률,3년생존률,5년생존률,1년생존률(신),3년생존률(신),5년생존률(신),최근10년영업기간,최근30년영업기간
0,0,202203,강남구,CS100001,한식,3.607718,1,116,145,2.2,2.7,71.8,46.9,35.2,77.3,50.1,34.3,2.7년,4.6년
1,1,202203,강동구,CS100001,한식,2.775424,0,53,54,2.3,2.3,73.9,46.9,37.6,75.1,52.9,32.2,2.6년,4.4년
2,2,202203,강북구,CS100001,한식,2.540466,0,39,47,1.9,2.3,70.3,45.3,36.5,74.3,49.2,32.3,2.6년,4.5년
3,3,202203,강서구,CS100001,한식,3.015158,1,96,103,3.1,3.3,72.0,45.5,33.6,74.7,49.7,32.6,2.5년,4.3년
4,4,202203,관악구,CS100001,한식,2.721897,0,53,79,2.0,3.0,66.7,44.3,32.4,72.0,53.6,31.4,2.6년,4.5년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,202212,용산구,CS100010,커피,3.025641,1,47,27,5.4,3.1,76.3,53.0,39.4,79.6,56.5,40.4,2.8년,3.6년
996,996,202212,은평구,CS100010,커피,2.659218,0,40,25,6.0,3.7,73.6,43.4,27.1,77.4,51.8,28.7,2.5년,3.2년
997,997,202212,종로구,CS100010,커피,3.205776,1,66,34,5.2,2.7,68.3,45.7,35.6,80.2,54.9,35.6,3년,4.2년
998,998,202212,중구,CS100010,커피,3.551020,1,41,41,3.7,3.7,75.2,51.0,37.9,82.8,50.2,38.3,2.8년,4.2년


# 분기, 업종, 구, 매출등급

In [5]:
df=등급_개폐업율추가
df

,Unnamed: 0,데이터기준년월,시군구명,서비스업종코드,서비스업종,매출등급,매출등급01,개업수,폐업수,개업률,폐업률,1년생존률,3년생존률,5년생존률,1년생존률(신),3년생존률(신),5년생존률(신),최근10년영업기간,최근30년영업기간
0,0,202203,강남구,CS100001,한식,3.607718,1,116,145,2.2,2.7,71.8,46.9,35.2,77.3,50.1,34.3,2.7년,4.6년
1,1,202203,강동구,CS100001,한식,2.775424,0,53,54,2.3,2.3,73.9,46.9,37.6,75.1,52.9,32.2,2.6년,4.4년
2,2,202203,강북구,CS100001,한식,2.540466,0,39,47,1.9,2.3,70.3,45.3,36.5,74.3,49.2,32.3,2.6년,4.5년
3,3,202203,강서구,CS100001,한식,3.015158,1,96,103,3.1,3.3,72.0,45.5,33.6,74.7,49.7,32.6,2.5년,4.3년
4,4,202203,관악구,CS100001,한식,2.721897,0,53,79,2.0,3.0,66.7,44.3,32.4,72.0,53.6,31.4,2.6년,4.5년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,202212,용산구,CS100010,커피,3.025641,1,47,27,5.4,3.1,76.3,53.0,39.4,79.6,56.5,40.4,2.8년,3.6년
996,996,202212,은평구,CS100010,커피,2.659218,0,40,25,6.0,3.7,73.6,43.4,27.1,77.4,51.8,28.7,2.5년,3.2년
997,997,202212,종로구,CS100010,커피,3.205776,1,66,34,5.2,2.7,68.3,45.7,35.6,80.2,54.9,35.6,3년,4.2년
998,998,202212,중구,CS100010,커피,3.551020,1,41,41,3.7,3.7,75.2,51.0,37.9,82.8,50.2,38.3,2.8년,4.2년


In [6]:
df.columns

Index(['Unnamed: 0', '데이터기준년월', '시군구명', '서비스업종코드', '서비스업종', '매출등급', '매출등급01',
       '개업수', '폐업수', '개업률', '폐업률', '1년생존률', '3년생존률', '5년생존률', '1년생존률(신)',
       '3년생존률(신)', '5년생존률(신)', '최근10년영업기간', '최근30년영업기간'],
      dtype='object')

# reindex() 함수로 칼럼 순서 재정비

In [7]:
df=df.reindex(columns=['데이터기준년월', '시군구명', '서비스업종코드', '서비스업종', '매출등급', '매출등급01', 
                       '개업수','폐업수', '개업률', '폐업률', '1년생존률', '3년생존률', '5년생존률', 
                       '1년생존률(신)','3년생존률(신)', '5년생존률(신)','최근10년영업기간','최근30년영업기간'])
df

,데이터기준년월,시군구명,서비스업종코드,서비스업종,매출등급,매출등급01,개업수,폐업수,개업률,폐업률,1년생존률,3년생존률,5년생존률,1년생존률(신),3년생존률(신),5년생존률(신),최근10년영업기간,최근30년영업기간
0,202203,강남구,CS100001,한식,3.607718,1,116,145,2.2,2.7,71.8,46.9,35.2,77.3,50.1,34.3,2.7년,4.6년
1,202203,강동구,CS100001,한식,2.775424,0,53,54,2.3,2.3,73.9,46.9,37.6,75.1,52.9,32.2,2.6년,4.4년
2,202203,강북구,CS100001,한식,2.540466,0,39,47,1.9,2.3,70.3,45.3,36.5,74.3,49.2,32.3,2.6년,4.5년
3,202203,강서구,CS100001,한식,3.015158,1,96,103,3.1,3.3,72.0,45.5,33.6,74.7,49.7,32.6,2.5년,4.3년
4,202203,관악구,CS100001,한식,2.721897,0,53,79,2.0,3.0,66.7,44.3,32.4,72.0,53.6,31.4,2.6년,4.5년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,202212,용산구,CS100010,커피,3.025641,1,47,27,5.4,3.1,76.3,53.0,39.4,79.6,56.5,40.4,2.8년,3.6년
996,202212,은평구,CS100010,커피,2.659218,0,40,25,6.0,3.7,73.6,43.4,27.1,77.4,51.8,28.7,2.5년,3.2년
997,202212,종로구,CS100010,커피,3.205776,1,66,34,5.2,2.7,68.3,45.7,35.6,80.2,54.9,35.6,3년,4.2년
998,202212,중구,CS100010,커피,3.551020,1,41,41,3.7,3.7,75.2,51.0,37.9,82.8,50.2,38.3,2.8년,4.2년


# 매출 데이터와 매핑을 위해 '서비스업종코드' => '서비스_업종_코드' 칼럼 이름 변경

In [8]:
df = df.rename(columns={'서비스업종코드': '서비스_업종_코드'})
df

,데이터기준년월,시군구명,서비스_업종_코드,서비스업종,매출등급,매출등급01,개업수,폐업수,개업률,폐업률,1년생존률,3년생존률,5년생존률,1년생존률(신),3년생존률(신),5년생존률(신),최근10년영업기간,최근30년영업기간
0,202203,강남구,CS100001,한식,3.607718,1,116,145,2.2,2.7,71.8,46.9,35.2,77.3,50.1,34.3,2.7년,4.6년
1,202203,강동구,CS100001,한식,2.775424,0,53,54,2.3,2.3,73.9,46.9,37.6,75.1,52.9,32.2,2.6년,4.4년
2,202203,강북구,CS100001,한식,2.540466,0,39,47,1.9,2.3,70.3,45.3,36.5,74.3,49.2,32.3,2.6년,4.5년
3,202203,강서구,CS100001,한식,3.015158,1,96,103,3.1,3.3,72.0,45.5,33.6,74.7,49.7,32.6,2.5년,4.3년
4,202203,관악구,CS100001,한식,2.721897,0,53,79,2.0,3.0,66.7,44.3,32.4,72.0,53.6,31.4,2.6년,4.5년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,202212,용산구,CS100010,커피,3.025641,1,47,27,5.4,3.1,76.3,53.0,39.4,79.6,56.5,40.4,2.8년,3.6년
996,202212,은평구,CS100010,커피,2.659218,0,40,25,6.0,3.7,73.6,43.4,27.1,77.4,51.8,28.7,2.5년,3.2년
997,202212,종로구,CS100010,커피,3.205776,1,66,34,5.2,2.7,68.3,45.7,35.6,80.2,54.9,35.6,3년,4.2년
998,202212,중구,CS100010,커피,3.551020,1,41,41,3.7,3.7,75.2,51.0,37.9,82.8,50.2,38.3,2.8년,4.2년


In [9]:
df['서비스_업종_코드'].unique()

array(['CS100001', 'CS100002', 'CS100003', 'CS100004', 'CS100005',
       'CS100006', 'CS100007', 'CS100008', 'CS100009', 'CS100010'],
      dtype=object)

In [10]:
df['서비스_업종_코드'].nunique()

10

# 매출 테이블 정리

In [11]:
상권추정매출= pd.read_csv("서울시 상권분석서비스(상권-추정매출2022).csv", encoding='euc-kr')
상권추정매출

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2022,4,D,발달상권,2120139,대림3동사거리,CS300031,가구,205619535,220,...,0,176,44,0,0,0,0,44,176,*
1,2022,4,D,발달상권,2120068,사가정역,CS100005,제과점,1111094837,97349,...,9275,33721,62371,1769,17108,23129,20604,18990,14492,5
2,2022,4,R,전통시장,2130322,고분다리전통시장,CS300018,의약품,168879417,10969,...,0,4939,6030,78,312,1223,1169,2496,5691,3
3,2022,4,D,발달상권,2120230,송파사거리(송파역),CS300017,시계및귀금속,47289011,156,...,0,59,97,0,0,15,0,38,103,5
4,2022,4,D,발달상권,2120234,가락시장역,CS200016,당구장,191548486,7142,...,1726,6518,495,136,812,1417,1156,1815,1678,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136815,2022,1,D,발달상권,2120210,선릉역,CS100001,한식음식점,23793250961,656818,...,33341,325540,198873,5225,92276,142044,117282,100271,67313,252
136816,2022,1,D,발달상권,2120210,선릉역,CS300022,화장품,16603436633,129219,...,4457,23784,105284,1676,41781,37758,22677,15331,9845,227
136817,2022,1,A,골목상권,2110229,장평중학교,CS100002,중식음식점,116283313,5785,...,12,3519,1646,14,329,1061,1602,1156,1005,4
136818,2022,1,D,발달상권,2120093,연희동,CS100010,커피-음료,2040140668,201894,...,2578,73439,123527,1536,66115,62268,30498,25224,11328,34


In [12]:
상권추정매출.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136820 entries, 0 to 136819
Data columns (total 80 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   기준_년_코드          136820 non-null  int64 
 1   기준_분기_코드         136820 non-null  int64 
 2   상권_구분_코드         136820 non-null  object
 3   상권_구분_코드_명       136820 non-null  object
 4   상권_코드            136820 non-null  int64 
 5   상권_코드_명          136820 non-null  object
 6   서비스_업종_코드        136820 non-null  object
 7   서비스_업종_코드_명      136820 non-null  object
 8   분기당_매출_금액        136820 non-null  int64 
 9   분기당_매출_건수        136820 non-null  int64 
 10  주중_매출_비율         136820 non-null  int64 
 11  주말_매출_비율         136820 non-null  int64 
 12  월요일_매출_비율        136820 non-null  int64 
 13  화요일_매출_비율        136820 non-null  int64 
 14  수요일_매출_비율        136820 non-null  int64 
 15  목요일_매출_비율        136820 non-null  int64 
 16  금요일_매출_비율        136820 non-null  int64 
 17  토요일_매출_비율 

# 매출 테이블에 시군구명을 매핑하기 위해 
# 상권_코드, 시군구_코드, 시군구명 매핑 작업

In [12]:
상권영역 = pd.read_csv("서울시 상권분석서비스(상권영역).csv", encoding="euc-kr")
상권영역.head()

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,202112,A,골목상권,2110753,시흥동 은행나무사거리,191934,438813,11545,11545710,NaN
1,202112,A,골목상권,2110427,노원 을지대학교병원,206265,459431,11350,11350611,NaN
2,202112,A,골목상권,2110755,동일여자고등학교,192300,439070,11545,11545680,NaN
3,202112,A,골목상권,2110739,시흥3동 우편취급국,191289,438014,11545,11545690,NaN
4,202112,A,골목상권,2111017,송파나루역 3번,210222,445516,11710,11710562,NaN


In [13]:
상권영역2=상권영역[['상권_코드','시군구_코드']]
상권영역2

,상권_코드,시군구_코드
0,2110753,11545
1,2110427,11350
2,2110755,11545
3,2110739,11545
4,2111017,11710
...,...,...
1666,1001493,11140
1667,1001494,11110
1668,1001495,11710
1669,1001496,11680


In [14]:
코드정보 = pd.read_csv("서울시 건축물대장 법정동 코드정보.csv", encoding='euc-kr')
코드정보

,시군구코드,법정동코드,행정동코드,시도명,시군구명,법정동명,행정동명,적용시작일,적용만료일
0,11680,11400,675,서울특별시,강남구,일원동,개포3동,20221223,99991231
1,11680,10600,675,서울특별시,강남구,대치동,개포3동,20221223,99991231
2,11680,10300,675,서울특별시,강남구,개포동,개포3동,20221223,99991231
3,11740,10300,525,서울특별시,강동구,상일동,상일제1동,20210701,99991231
4,11740,10300,526,서울특별시,강동구,상일동,상일제2동,20210701,99991231
...,...,...,...,...,...,...,...,...,...
2350,11110,13600,610,서울특별시,종로구,인사동,종로1.2가동,19880423,19981201
2351,11110,13700,610,서울특별시,종로구,낙원동,종로1.2가동,19880423,19981201
2352,11110,13800,610,서울특별시,종로구,종로2가,종로1.2가동,19880423,19981201
2353,11110,15000,620,서울특별시,종로구,훈정동,종로3.4가동,19880423,19981201


In [15]:
구 = 코드정보[["시군구코드", "시군구명"]].drop_duplicates('시군구코드', keep='first')
구

,시군구코드,시군구명
0,11680,강남구
3,11740,강동구
5,11290,성북구
6,11530,구로구
7,11305,강북구
13,11410,서대문구
14,11500,강서구
18,11440,마포구
19,11710,송파구
23,11230,동대문구


In [16]:
# 시군구 코드에 11000 이라는 의미없는 값이 들어가 있어서 제거해줌
구 = 구[구["시군구코드"] != 11000]
구

,시군구코드,시군구명
0,11680,강남구
3,11740,강동구
5,11290,성북구
6,11530,구로구
7,11305,강북구
13,11410,서대문구
14,11500,강서구
18,11440,마포구
19,11710,송파구
23,11230,동대문구


In [17]:
구=구.rename(columns={'시군구코드': '시군구_코드'})
구

,시군구_코드,시군구명
0,11680,강남구
3,11740,강동구
5,11290,성북구
6,11530,구로구
7,11305,강북구
13,11410,서대문구
14,11500,강서구
18,11440,마포구
19,11710,송파구
23,11230,동대문구


In [18]:
구_상권코드=상권영역2.merge(구, on='시군구_코드')
구_상권코드

,상권_코드,시군구_코드,시군구명
0,2110753,11545,금천구
1,2110755,11545,금천구
2,2110739,11545,금천구
3,2110734,11545,금천구
4,2110751,11545,금천구
...,...,...,...
1666,2130268,11590,동작구
1667,2130271,11590,동작구
1668,2130270,11590,동작구
1669,2130269,11590,동작구


# 매출 테이블 - 구 매핑

In [19]:
상권추정매출

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2022,4,D,발달상권,2120139,대림3동사거리,CS300031,가구,205619535,220,...,0,176,44,0,0,0,0,44,176,*
1,2022,4,D,발달상권,2120068,사가정역,CS100005,제과점,1111094837,97349,...,9275,33721,62371,1769,17108,23129,20604,18990,14492,5
2,2022,4,R,전통시장,2130322,고분다리전통시장,CS300018,의약품,168879417,10969,...,0,4939,6030,78,312,1223,1169,2496,5691,3
3,2022,4,D,발달상권,2120230,송파사거리(송파역),CS300017,시계및귀금속,47289011,156,...,0,59,97,0,0,15,0,38,103,5
4,2022,4,D,발달상권,2120234,가락시장역,CS200016,당구장,191548486,7142,...,1726,6518,495,136,812,1417,1156,1815,1678,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136815,2022,1,D,발달상권,2120210,선릉역,CS100001,한식음식점,23793250961,656818,...,33341,325540,198873,5225,92276,142044,117282,100271,67313,252
136816,2022,1,D,발달상권,2120210,선릉역,CS300022,화장품,16603436633,129219,...,4457,23784,105284,1676,41781,37758,22677,15331,9845,227
136817,2022,1,A,골목상권,2110229,장평중학교,CS100002,중식음식점,116283313,5785,...,12,3519,1646,14,329,1061,1602,1156,1005,4
136818,2022,1,D,발달상권,2120093,연희동,CS100010,커피-음료,2040140668,201894,...,2578,73439,123527,1536,66115,62268,30498,25224,11328,34


In [20]:
구_상권매출=상권추정매출.merge(구_상권코드, on=['상권_코드'])
구_상권매출

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수,시군구_코드,시군구명
0,2022,4,D,발달상권,2120139,대림3동사거리,CS300031,가구,205619535,220,...,44,0,0,0,0,44,176,*,11560,영등포구
1,2022,4,D,발달상권,2120139,대림3동사거리,CS200026,자동차미용,13360413,346,...,61,0,8,61,69,77,53,3,11560,영등포구
2,2022,4,D,발달상권,2120139,대림3동사거리,CS200037,노래방,174237907,1071,...,151,0,45,198,135,460,158,12,11560,영등포구
3,2022,4,D,발달상권,2120139,대림3동사거리,CS300021,문구,126766179,11520,...,5120,173,558,1636,3515,3966,1672,5,11560,영등포구
4,2022,4,D,발달상권,2120139,대림3동사거리,CS200001,일반교습학원,2167240118,4514,...,2751,0,0,2037,2266,200,11,9,11560,영등포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136815,2022,2,A,골목상권,2110443,수색역 1번,CS200028,미용실,19748204,753,...,484,0,14,129,170,327,113,*,11380,은평구
136816,2022,2,A,골목상권,2110443,수색역 1번,CS100001,한식음식점,1636212,77,...,12,0,46,0,0,6,12,*,11380,은평구
136817,2022,1,A,골목상권,2110443,수색역 1번,CS100001,한식음식점,362022,6,...,0,0,0,0,0,6,0,*,11380,은평구
136818,2022,1,A,골목상권,2110443,수색역 1번,CS200028,미용실,18888309,629,...,331,0,50,105,172,240,62,*,11380,은평구


# 년,분기 매핑

In [21]:
구_상권매출.loc[구_상권매출['기준_분기_코드'] == 1, '데이터기준년월'] = 202203
구_상권매출.loc[구_상권매출['기준_분기_코드'] == 2, '데이터기준년월'] = 202206
구_상권매출.loc[구_상권매출['기준_분기_코드'] == 3, '데이터기준년월'] = 202209
구_상권매출.loc[구_상권매출['기준_분기_코드'] == 4, '데이터기준년월'] = 202212

구_상권매출['데이터기준년월']=구_상권매출['데이터기준년월'].astype(int)
구_상권매출=구_상권매출.astype(str)

In [22]:
df2=구_상권매출
df2

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수,시군구_코드,시군구명,데이터기준년월
0,2022,4,D,발달상권,2120139,대림3동사거리,CS300031,가구,205619535,220,...,0,0,0,0,44,176,*,11560,영등포구,202212
1,2022,4,D,발달상권,2120139,대림3동사거리,CS200026,자동차미용,13360413,346,...,0,8,61,69,77,53,3,11560,영등포구,202212
2,2022,4,D,발달상권,2120139,대림3동사거리,CS200037,노래방,174237907,1071,...,0,45,198,135,460,158,12,11560,영등포구,202212
3,2022,4,D,발달상권,2120139,대림3동사거리,CS300021,문구,126766179,11520,...,173,558,1636,3515,3966,1672,5,11560,영등포구,202212
4,2022,4,D,발달상권,2120139,대림3동사거리,CS200001,일반교습학원,2167240118,4514,...,0,0,2037,2266,200,11,9,11560,영등포구,202212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136815,2022,2,A,골목상권,2110443,수색역 1번,CS200028,미용실,19748204,753,...,0,14,129,170,327,113,*,11380,은평구,202206
136816,2022,2,A,골목상권,2110443,수색역 1번,CS100001,한식음식점,1636212,77,...,0,46,0,0,6,12,*,11380,은평구,202206
136817,2022,1,A,골목상권,2110443,수색역 1번,CS100001,한식음식점,362022,6,...,0,0,0,0,6,0,*,11380,은평구,202203
136818,2022,1,A,골목상권,2110443,수색역 1번,CS200028,미용실,18888309,629,...,0,50,105,172,240,62,*,11380,은평구,202203


In [23]:
df2.columns

Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '서비스_업종_코드', '서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수', '주중_매출_비율',
       '주말_매출_비율', '월요일_매출_비율', '화요일_매출_비율', '수요일_매출_비율', '목요일_매출_비율',
       '금요일_매출_비율', '토요일_매출_비율', '일요일_매출_비율', '시간대_00~06_매출_비율',
       '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', '시간대_14~17_매출_비율',
       '시간대_17~21_매출_비율', '시간대_21~24_매출_비율', '남성_매출_비율', '여성_매출_비율',
       '연령대_10_매출_비율', '연령대_20_매출_비율', '연령대_30_매출_비율', '연령대_40_매출_비율',
       '연령대_50_매출_비율', '연령대_60_이상_매출_비율', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액',
       '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
       '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
       '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수',
       '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수',

In [24]:
df

,데이터기준년월,시군구명,서비스_업종_코드,서비스업종,매출등급,매출등급01,개업수,폐업수,개업률,폐업률,1년생존률,3년생존률,5년생존률,1년생존률(신),3년생존률(신),5년생존률(신),최근10년영업기간,최근30년영업기간
0,202203,강남구,CS100001,한식,3.607718,1,116,145,2.2,2.7,71.8,46.9,35.2,77.3,50.1,34.3,2.7년,4.6년
1,202203,강동구,CS100001,한식,2.775424,0,53,54,2.3,2.3,73.9,46.9,37.6,75.1,52.9,32.2,2.6년,4.4년
2,202203,강북구,CS100001,한식,2.540466,0,39,47,1.9,2.3,70.3,45.3,36.5,74.3,49.2,32.3,2.6년,4.5년
3,202203,강서구,CS100001,한식,3.015158,1,96,103,3.1,3.3,72.0,45.5,33.6,74.7,49.7,32.6,2.5년,4.3년
4,202203,관악구,CS100001,한식,2.721897,0,53,79,2.0,3.0,66.7,44.3,32.4,72.0,53.6,31.4,2.6년,4.5년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,202212,용산구,CS100010,커피,3.025641,1,47,27,5.4,3.1,76.3,53.0,39.4,79.6,56.5,40.4,2.8년,3.6년
996,202212,은평구,CS100010,커피,2.659218,0,40,25,6.0,3.7,73.6,43.4,27.1,77.4,51.8,28.7,2.5년,3.2년
997,202212,종로구,CS100010,커피,3.205776,1,66,34,5.2,2.7,68.3,45.7,35.6,80.2,54.9,35.6,3년,4.2년
998,202212,중구,CS100010,커피,3.551020,1,41,41,3.7,3.7,75.2,51.0,37.9,82.8,50.2,38.3,2.8년,4.2년


# df 와 df2 결합을 위해 문자 타입으로 변경

In [25]:
df=df.astype(str)
df2=df2.astype(str)

# df3 = df 와 df2 merge (기준 값: df의 '데이터기준년월', '시군구명', '서비스_업종_코드')

In [26]:
df3 = df.merge(df2, on=['데이터기준년월', '시군구명', '서비스_업종_코드'], how='left')

In [27]:
df3

,데이터기준년월,시군구명,서비스_업종_코드,서비스업종,매출등급,매출등급01,개업수,폐업수,개업률,폐업률,...,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수,시군구_코드
0,202203,강남구,CS100001,한식,3.607717934480844,1,116,145,2.2,2.7,...,2410,3665,0,971,2520,1320,763,502,5,11680
1,202203,강남구,CS100001,한식,3.607717934480844,1,116,145,2.2,2.7,...,71669,45424,411,22326,36148,25579,19526,13097,90,11680
2,202203,강남구,CS100001,한식,3.607717934480844,1,116,145,2.2,2.7,...,20848,13583,1904,2851,6489,11369,8908,2904,33,11680
3,202203,강남구,CS100001,한식,3.607717934480844,1,116,145,2.2,2.7,...,66802,37861,516,25510,36477,22180,13662,6316,91,11680
4,202203,강남구,CS100001,한식,3.607717934480844,1,116,145,2.2,2.7,...,325540,198873,5225,92276,142044,117282,100271,67313,252,11680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35848,202212,중랑구,CS100010,커피,2.590062111801242,0,42,23,8.2,4.5,...,1096,852,0,302,712,139,307,488,3,11260
35849,202212,중랑구,CS100010,커피,2.590062111801242,0,42,23,8.2,4.5,...,13143,13598,1743,10178,4626,4976,4563,654,16,11260
35850,202212,중랑구,CS100010,커피,2.590062111801242,0,42,23,8.2,4.5,...,810,1713,9,324,207,594,939,450,7,11260
35851,202212,중랑구,CS100010,커피,2.590062111801242,0,42,23,8.2,4.5,...,794,2057,15,1263,872,366,244,91,11,11260


In [28]:
df3['서비스_업종_코드'].unique()

array(['CS100001', 'CS100002', 'CS100003', 'CS100004', 'CS100005',
       'CS100006', 'CS100007', 'CS100008', 'CS100009', 'CS100010'],
      dtype=object)

In [29]:
df3['서비스업종'].unique()

array(['한식', '중식', '일식', '기타 외국식', '제과제빵', '패스트푸드', '치킨', '분식', '주점',
       '커피'], dtype=object)

In [30]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35853 entries, 0 to 35852
Data columns (total 98 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   데이터기준년월          35853 non-null  object
 1   시군구명             35853 non-null  object
 2   서비스_업종_코드        35853 non-null  object
 3   서비스업종            35853 non-null  object
 4   매출등급             35853 non-null  object
 5   매출등급01           35853 non-null  object
 6   개업수              35853 non-null  object
 7   폐업수              35853 non-null  object
 8   개업률              35853 non-null  object
 9   폐업률              35853 non-null  object
 10  1년생존률            35853 non-null  object
 11  3년생존률            35853 non-null  object
 12  5년생존률            35853 non-null  object
 13  1년생존률(신)         35853 non-null  object
 14  3년생존률(신)         35853 non-null  object
 15  5년생존률(신)         35853 non-null  object
 16  최근10년영업기간        35853 non-null  object
 17  최근30년영업기간        35853 non-null

# CSV 저장 (기본df_매출df_합)

In [31]:
df3.to_csv("기본df_매출df_합.csv", encoding='ansi')